# CAI - Rebalancing data for February 2023 

## Setting

In [228]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy, gmx_token_contract


infura_url = decouple.config("AVALANCHE_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

In [229]:
from pycoingecko import CoinGeckoAPI
key = decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [230]:
import plotly.express as px
import plotly.graph_objects as go

In [231]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. Any token included in our index must meet all the criteria below:
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [232]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [233]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

# Adding AVAX
coins['avalanche-2'] = {'id': 'avalanche-2','symbol':normalize_symb('avax'), 'name':'Avalanche','addr':'0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7',}


##### Market data

In [234]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

cg_tokens_data = cg_tokens_data.append(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'),ignore_index=False)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_92367/3118279002.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [235]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [236]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]


In [237]:
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])


In [238]:
tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]
tokens_market_data.head()

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
tether,Tether,USDT,6.693922e+10,3.0,1.00000,6.685438e+10,6.685438e+10,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7
usd-coin,USD Coin,USDC,4.364956e+10,4.0,0.99908,4.365950e+10,4.366338e+10,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e
binance-usd,Binance USD,BUSD,1.547772e+10,7.0,1.00100,1.544935e+10,1.544935e+10,0x9c9e5fd8bbc25984b178fdce6117defa39d2db39
avalanche-2,Avalanche,AVAX,5.336285e+09,18.0,16.91000,4.199587e+08,3.145527e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
dai,Dai,DAI,5.147174e+09,19.0,1.00100,5.156659e+09,5.153087e+09,0xd586e7f844cea2f87f50152665bcbc2c279d8d70


In [239]:
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

### Token's supply check 

In [240]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]

### Retrieve historical data & token age check

In [241]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in sp_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 6 * 30:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {6*30} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 6 * 30:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {6 * 30} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Note: BTCB, marketcap data available only for 99 < 180 days
Excluding XEN, prices data available only for 106 < 180 days
Excluding GENI, prices data available only for 22 < 180 days
Note: CLY, marketcap data available only for 144 < 180 days
Note: 3ULL, marketcap data available only for 123 < 180 days
Note: BUILD, marketcap data available only for 151 < 180 days
Note: MAGIC, marketcap data available only for 13 < 180 days
Excluding DCAR, prices data available only for 168 < 180 days
Excluding CAI, prices data available only for 140 < 180 days
Note: CHRO, marketcap data available only for 78 < 180 days


### Non quantative checks

In [242]:
manual_exclusions = [
    {
        'gecko_id': 'tether',
        'reason': 'stablecoin'
    },
    {
        'gecko_id': 'usd-coin',
        'reason': 'stablecoin'
    },
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'wrapped-bitcoin',
        'reason' : 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'dai',
        'reason' : 'stablecoin'
    },
    {
        'gecko_id': 'bitcoin-avalanche-bridged-btc-b',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'woo-network',
        'reason': 'Bridged asset'
    }
]
for token in manual_exclusions:
    exclude_list.append(token['gecko_id'])

exclude_list

['xen-crypto',
 'genius',
 'dragon-crypto-argenti',
 'colony-avalanche-index',
 'tether',
 'usd-coin',
 'spell-token',
 'stacktical',
 'zookeeper',
 'pendle',
 'everrise',
 'wrapped-bitcoin',
 'dai',
 'bitcoin-avalanche-bridged-btc-b']

In [243]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list')
passed_q_and_non_q_checks

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
binance-usd,Binance USD,BUSD,1.547772e+10,7.0,1.001000,1.544935e+10,1.544935e+10,0x9c9e5fd8bbc25984b178fdce6117defa39d2db39
avalanche-2,Avalanche,AVAX,5.336285e+09,18.0,16.910000,4.199587e+08,3.145527e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
uniswap,Uniswap,UNI,4.710524e+09,20.0,6.240000,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
chainlink,Chainlink,LINK,3.279451e+09,24.0,6.650000,1.000000e+09,4.916000e+08,0x5947bb275c521040051d82396192181b413227a3
aave,Aave,AAVE,1.141600e+09,44.0,79.890000,1.600000e+07,1.423718e+07,0x63a72806098bd3d9520cc43356dd78afe5d386d9
...,...,...,...,...,...,...,...,...
morphie,Morphie,MRFI,1.305087e+04,3662.0,0.000002,2.000000e+10,8.069296e+09,0x05e481b19129b560e921e487adb281e70bdba463
fortressdao,Fortress,FORT,1.183678e+04,3674.0,0.008443,1.409126e+06,1.401945e+06,0xf6d46849db378ae01d93732585bec2c4480d1fd5
ape-in,Ape In,APEIN,1.045318e+04,3692.0,0.280070,3.750000e+04,3.750000e+04,0x938fe3788222a74924e062120e7bfac829c719fb


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [244]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5

In [245]:
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

In [246]:
url_0x = "https://avalanche.api.0x.org/swap/v1/quote"

In [247]:
decimals = 18
slippages = []
for id, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': data['addr'],
            'sellToken': 'AVAX',
            'sellAmount': int(trade_value_tiny / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals)
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'id': id,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        continue
    

In [248]:
slippages_df = pd.DataFrame(slippages)
slippages_df

,id,spot_price,delivery_price,slippage
0,binance-usd,1.682635e+01,9.923184,-0.410259
1,avalanche-2,1.000000e+00,1.000000,0.000000
2,uniswap,1.649058e+00,0.004654,-0.997178
3,chainlink,2.535195e+00,2.097841,-0.172513
4,aave,2.119147e-01,0.133798,-0.368625
...,...,...,...,...
102,morphie,1.636710e+07,80085.635828,-0.995107
103,fortressdao,1.953676e+03,164.412811,-0.915844
104,ape-in,5.962494e+01,5.586114,-0.906312
105,impermax,8.824193e+04,355.415065,-0.995972


In [249]:
include_list = list(slippages_df[abs(slippages_df['slippage']) < abs(max_slippage)]['id'])
qualified_assets = passed_q_and_non_q_checks.query('index in @include_list')
qualified_assets


,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,5.336285e+09,18.0,16.910000,4.199587e+08,3.145527e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
gmx,GMX,GMX,4.272736e+07,97.0,48.860000,8.404006e+06,8.403957e+06,0x62edc0692bd897d2295872a9ffcac5425011c661
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,9.311953e+07,271.0,17.990000,5.176327e+06,5.176326e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,6.199367e+07,363.0,0.187827,4.645131e+08,3.302313e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,2.573977e+07,612.0,0.007826,7.200000e+09,3.288188e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5


### Marketcap ranking & filtering

In [250]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [251]:
# exclude savax from weighting since it should be a part of avax weight
if 'benqi-liquid-staked-avax' in chosen_tokens.index:
    chosen_tokens_ex_savax = chosen_tokens.drop('benqi-liquid-staked-avax')
    weights = chosen_tokens_ex_savax['market_cap'].div(chosen_tokens_ex_savax['market_cap'].sum())
else:
    weights = chosen_tokens['market_cap'].div(chosen_tokens['market_cap'].sum())



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [252]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum()
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum()).mul(c)
    
    return w

In [253]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum())
    
    return w

In [254]:
adjusted_weights = adjust_weights(weights, max_weight=0.5) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.5)

### Split AVAX 

In [255]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    split = 0.6
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - split)

adjusted_weights =  adjusted_weights.sort_values(ascending = False)
adjusted_weights


id
benqi-liquid-staked-avax    0.300000
joe                         0.237595
avalanche-2                 0.200000
gmx                         0.163756
benqi                       0.098649
Name: market_cap, dtype: float64

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [256]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [257]:
converted_weights = convert_weights(adjusted_weights)

## Summary table

In [258]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = chosen_tokens['market_cap']
cai['price'] = chosen_tokens['current_price']
cai['weight'] = adjusted_weights
cai['weight_converted'] = converted_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,5.336285e+09,16.910000,0.200000,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,9.311953e+07,17.990000,0.300000,77,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,6.199367e+07,0.187827,0.237595,61,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
gmx,GMX,GMX,4.272736e+07,48.860000,0.163756,41,0x62edc0692bd897d2295872a9ffcac5425011c661
benqi,BENQI,QI,2.573977e+07,0.007826,0.098649,25,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5


In [259]:
#Querying current contract to ascertain which assets will be dropped

cai_token_address = '0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0'
ia_contract = web3_object.eth.contract(address=cai_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()


for address in addresses:
    address = address.lower()
    if address not in list(cai['address']):
        data = tokens_market_data[tokens_market_data['addr']== address]
        cai.loc[data.index[0]] = [data['name'][0],data['symbol'][0],data['market_cap'][0],data['current_price'][0],0,0,address]

cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,5.336285e+09,16.910000,0.200000,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,9.311953e+07,17.990000,0.300000,77,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,6.199367e+07,0.187827,0.237595,61,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
gmx,GMX,GMX,4.272736e+07,48.860000,0.163756,41,0x62edc0692bd897d2295872a9ffcac5425011c661
benqi,BENQI,QI,2.573977e+07,0.007826,0.098649,25,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5


In [260]:
zipped_assets = list(zip(list(cai['address']),list(cai['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x62edc0692bd897d2295872a9ffcac5425011c661,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
[77, 41, 61, 25, 51]
